In [90]:
import pandas as pd
import json
import psycopg2 as p2
from datetime import date,datetime,timedelta

In [2]:
rapid_reorder_control=pd.read_csv('data\Rapid_Reorder_Control_export.csv')
rapid_reorder_recipients=pd.read_csv('data\Rapid_Reorder_Recipients_export.csv')

In [3]:
def aws_connection(file_name:str=''):
    # Connecting to Amazon Redshift DB
    # authentication credentials to connect to redshift db from json file
    with open('./cred/{fl_name}.json'.format(fl_name = file_name)) as redshift_connection:
        aws_creds = json.load(redshift_connection)

    # connect to redshift:
    con = p2.connect(
            dbname= aws_creds['dbname'],
            host= aws_creds['host'],
            port= aws_creds['port'],
            user= aws_creds['user'],
            password= aws_creds['password'])
    return con

In [4]:
def query_aws(file_name: str='', query_path: str='./queries/', query_name: str = ''):
    '''Returns DataFrame based on a provided query '''
    conn = aws_connection(file_name=file_name)
    query = open(query_path + query_name).read()
    data = pd.read_sql(sql=query, con=conn)
    return data

### Reformating the control group, joined with the orders for the control group customers

In [5]:
all_orders_from_2019=query_aws(file_name='the_phoenix_connection', query_name='rapid_reorder_eda.sql')

In [6]:
controL_group_orders=pd.merge(all_orders_from_2019,rapid_reorder_control,how='inner',left_on='email',right_on='email')
controL_group_orders['unsubscribed_from_emails_at']=controL_group_orders['unsubscribed_from_emails_at'].apply(lambda x: x if pd.isna(x) else datetime.strptime(x[:-4],'%Y-%m-%d %H:%M:%S') - timedelta(hours=8))
controL_group_orders['opted_in_to_emails_at']=controL_group_orders['opted_in_to_emails_at'].apply(lambda x: x if pd.isna(x) else datetime.strptime(x[:-4],'%Y-%m-%d %H:%M:%S') - timedelta(hours=8))

In [7]:
test_group_orders=pd.merge(all_orders_from_2019,rapid_reorder_recipients,how='inner',left_on='email',right_on='email')
test_group_orders['unsubscribed_from_emails_at']=test_group_orders['unsubscribed_from_emails_at'].apply(lambda x: x if pd.isna(x) else datetime.strptime(x[:-4],'%Y-%m-%d %H:%M:%S') - timedelta(hours=8))
test_group_orders['opted_in_to_emails_at']=test_group_orders['opted_in_to_emails_at'].apply(lambda x: x if pd.isna(x) else datetime.strptime(x[:-4],'%Y-%m-%d %H:%M:%S') - timedelta(hours=8))

### Bringing promo codes into the picture

In [8]:
promo_codes=query_aws(file_name='the_phoenix_connection', query_name='rapid_reorder_promocodes.sql')

In [9]:
control_group_orders_with_promo_codes=pd.merge(controL_group_orders,promo_codes,how='left',on='promo_code_id')
test_group_orders_with_promo_codes=pd.merge(test_group_orders,promo_codes,how='left',on='promo_code_id')

#### Filtering only the promo codes we want to analyze: 
###### 'PIZZAGAIN','ROUNDTWO','ONEMORESLICE','STILLHUNGRY','ONEMOREPIE','ONCEAGAIN','NEVERENOUGH','REPEATZZA2','REPEATZZA5','DOUBLEDOWN'

In [10]:
control_group_focus_promo_codes = control_group_orders_with_promo_codes[control_group_orders_with_promo_codes['promo_code_tag'].isin(['PIZZAGAIN','ROUNDTWO',
                                                                                                                                      'ONEMORESLICE','STILLHUNGRY',
                                                                                                                                      'ONEMOREPIE','ONCEAGAIN',
                                                                                                                                      'NEVERENOUGH','REPEATZZA2',
                                                                                                                                      'REPEATZZA5','DOUBLEDOWN'])]
test_group_focus_promo_codes = test_group_orders_with_promo_codes[test_group_orders_with_promo_codes['promo_code_tag'].isin(['PIZZAGAIN','ROUNDTWO','ONEMORESLICE',
                                                                                                                             'STILLHUNGRY','ONEMOREPIE','ONCEAGAIN',
                                                                                                                             'NEVERENOUGH','REPEATZZA2','REPEATZZA5',
                                                                                                                             'DOUBLEDOWN'])]

In [11]:
test_group_focus_promo_codes=test_group_focus_promo_codes[['orders_id', 'customer', 'source', 'shop_id', 'date_purchased_pst',
       'mypizza_fees', 'cc_fees', 'email', 'cached_total', 'promo_code_id', 'user_id', 'unsubscribed_from_emails_at',
       'opted_in_to_emails_at', 'promo_code_tag', 'description', 'department', 'promo_flat_amount']]

##### adding days_between_opt_in_and_order

In [12]:
test_group_focus_promo_codes['days_between_opt_in_and_order']=test_group_focus_promo_codes['date_purchased_pst'] - test_group_focus_promo_codes['opted_in_to_emails_at']
control_group_focus_promo_codes['days_between_opt_in_and_order']=control_group_focus_promo_codes['date_purchased_pst'] - control_group_focus_promo_codes['opted_in_to_emails_at']

C:\Users\asaid\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


##### adding days_between_order_and_unsubscribed_from_emails_at

In [26]:
test_group_focus_promo_codes['days_between_order_and_unsubscribed_from_emails_at']=test_group_focus_promo_codes['date_purchased_pst'] - test_group_focus_promo_codes['unsubscribed_from_emails_at']
control_group_focus_promo_codes['days_between_order_and_unsubscribed_from_emails_at']=control_group_focus_promo_codes['date_purchased_pst'] - control_group_focus_promo_codes['unsubscribed_from_emails_at']

C:\Users\asaid\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Summmary for the data that we have

In [13]:
input_summary=pd.DataFrame(columns=['control_vs_test','Unique_users','Users_that_opted_in_to_emails','percent_of_user_that_opted_in'])
inputs=pd.DataFrame(columns=['control_vs_test','Unique_users','Users_that_opted_in_to_emails'])
inputs=pd.DataFrame([
    ['control', rapid_reorder_control['email'].nunique(), rapid_reorder_control['opted_in_to_emails_at'].notnull().sum(),
                      rapid_reorder_control['opted_in_to_emails_at'].notnull().sum()/rapid_reorder_control['email'].nunique()],
                     ['test', rapid_reorder_recipients['email'].nunique(), rapid_reorder_recipients['opted_in_to_emails_at'].notnull().sum(),
                      rapid_reorder_recipients['opted_in_to_emails_at'].notnull().sum()/rapid_reorder_recipients['email'].nunique()]
                    ]
    ,columns=['control_vs_test','Unique_users','Users_that_opted_in_to_emails','percent_of_user_that_opted_in'])
input_summary = input_summary.append(inputs,ignore_index=True)
input_summary

,control_vs_test,Unique_users,Users_that_opted_in_to_emails,percent_of_user_that_opted_in
0,control,339443,25398,0.074823
1,test,333551,25442,0.076276


## Getting summary based on promo code and time

### within 4 days after opt-in

In [14]:
four_days = timedelta(days=4)
zero_days = timedelta(days=0)

In [15]:
test_group_focus_promo_codes_within_4_days_after_opt_in = test_group_focus_promo_codes[(test_group_focus_promo_codes['days_between_opt_in_and_order'] < four_days)
                             &
                            (test_group_focus_promo_codes['days_between_opt_in_and_order'] > zero_days)]

control_group_focus_promo_codes_within_4_days_after_opt_in = control_group_focus_promo_codes[(control_group_focus_promo_codes['days_between_opt_in_and_order'] < four_days)
                             &
                            (control_group_focus_promo_codes['days_between_opt_in_and_order'] > zero_days)]

In [128]:
test_four_days_after_opt_in = test_group_focus_promo_codes_within_4_days_after_opt_in.groupby('promo_code_tag').customer.nunique().to_frame().reset_index().rename(columns={'customer':'test_four_days_after_opt_in'})
control_four_days_after_opt_in = control_group_focus_promo_codes_within_4_days_after_opt_in.groupby('promo_code_tag').customer.nunique().to_frame().reset_index().rename(columns={'customer':'control_four_days_after_opt_in'})

### after 4 days since opt-in

In [18]:
test_group_focus_promo_codes_after_4_days_after_opt_in = test_group_focus_promo_codes[(test_group_focus_promo_codes['days_between_opt_in_and_order'] > four_days)]

control_group_focus_promo_codes_after_4_days_after_opt_in = control_group_focus_promo_codes[(control_group_focus_promo_codes['days_between_opt_in_and_order'] > four_days)]

In [129]:
test_within_four_days_since_opt_in = test_group_focus_promo_codes_after_4_days_after_opt_in.groupby('promo_code_tag').customer.nunique().to_frame().reset_index().rename(columns={'customer':'test_within_four_days_since_opt_in'})
control_within_four_days_since_opt_in = control_group_focus_promo_codes_after_4_days_after_opt_in.groupby('promo_code_tag').customer.nunique().to_frame().reset_index().rename(columns={'customer':'control_within_four_days_since_opt_in'})

### before opt-in

In [21]:
test_group_focus_promo_codes_before_opt_in = test_group_focus_promo_codes[(test_group_focus_promo_codes['days_between_opt_in_and_order'] < zero_days)]

control_group_focus_promo_codes_before_opt_in = control_group_focus_promo_codes[(control_group_focus_promo_codes['days_between_opt_in_and_order'] < zero_days)]

In [130]:
test_before_opt_in=test_group_focus_promo_codes_before_opt_in.groupby('promo_code_tag').customer.nunique().to_frame().reset_index().rename(columns={'customer':'test_before_opt_in'})
control_before_opt_in=control_group_focus_promo_codes_before_opt_in.groupby('promo_code_tag').customer.nunique().to_frame().reset_index().rename(columns={'customer':'control_before_opt_in'})

### After a customer unsubscribed_from_emails_at

In [38]:
test_group_focus_promo_codes_after_unsubscribing = test_group_focus_promo_codes[(test_group_focus_promo_codes['days_between_order_and_unsubscribed_from_emails_at'] > zero_days)]

control_group_focus_promo_codes_after_unsubscribing = control_group_focus_promo_codes[(control_group_focus_promo_codes['days_between_order_and_unsubscribed_from_emails_at'] > zero_days)]

In [131]:
test_after_unsubscribing = test_group_focus_promo_codes_after_unsubscribing.groupby('promo_code_tag').customer.nunique().to_frame().reset_index().rename(columns={'customer':'test_after_unsubscribing'})
control_after_unsubscribing = control_group_focus_promo_codes_after_unsubscribing.groupby('promo_code_tag').customer.nunique().to_frame().reset_index().rename(columns={'customer':'control_after_unsubscribing'})

In [111]:
y=control_four_days_after_opt_in.rename(columns={'customer':'control_four_days_after_opt_in'})

In [114]:
result=pd.concat([x,y],axis=1,join='outer')

In [116]:
result=pd.merge(test_four_days_after_opt_in,control_four_days_after_opt_in,how='outer',on='promo_code_tag')

In [134]:
result=pd.merge(test_four_days_after_opt_in,control_four_days_after_opt_in, how='outer', on='promo_code_tag')
result=pd.merge(result, test_within_four_days_since_opt_in, how='outer', on='promo_code_tag')
result=pd.merge(result, control_within_four_days_since_opt_in, how='outer', on='promo_code_tag')
result=pd.merge(result, test_before_opt_in, how='outer', on='promo_code_tag')
result=pd.merge(result, control_before_opt_in, how='outer', on='promo_code_tag')
result=pd.merge(result, test_after_unsubscribing, how='outer', on='promo_code_tag')
result=pd.merge(result, control_after_unsubscribing, how='outer', on='promo_code_tag')

In [135]:
result

,promo_code_tag,test_four_days_after_opt_in,control_four_days_after_opt_in,test_within_four_days_since_opt_in,control_within_four_days_since_opt_in,test_before_opt_in,control_before_opt_in,test_after_unsubscribing,control_after_unsubscribing
0,DOUBLEDOWN,10,NaN,56,1.0,106,NaN,5,NaN
1,NEVERENOUGH,13,NaN,50,NaN,148,5.0,9,2.0
2,ONCEAGAIN,3,NaN,50,NaN,102,1.0,4,NaN
3,ONEMOREPIE,14,NaN,57,NaN,121,NaN,4,NaN
4,ONEMORESLICE,15,NaN,63,NaN,130,2.0,5,NaN
5,PIZZAGAIN,15,NaN,64,NaN,146,NaN,8,NaN
6,REPEATZZA2,10,NaN,48,1.0,120,2.0,7,NaN
7,REPEATZZA5,26,1.0,50,1.0,166,4.0,7,1.0
8,ROUNDTWO,18,NaN,48,NaN,112,2.0,5,NaN
9,STILLHUNGRY,14,NaN,41,NaN,111,2.0,2,NaN


In [125]:
x

[  promo_code_tag  customer
 0     DOUBLEDOWN        10
 1    NEVERENOUGH        13
 2      ONCEAGAIN         3
 3     ONEMOREPIE        14
 4   ONEMORESLICE        15
 5      PIZZAGAIN        15
 6     REPEATZZA2        10
 7     REPEATZZA5        26
 8       ROUNDTWO        18
 9    STILLHUNGRY        14,   promo_code_tag  customer
 0     REPEATZZA5         1]